Learning an unkown distribution where we have control over the sample placement
--------------------------------------------------------------------------------------------------------------------------

In the syle of Vorba et al. from "On-line Learning of Parametric Mixture Models for Light Transport Simulation" (2014).

In the paper, the unkown distribution is the incident radiation (radiance) Li(x). It is modeled by a learned gaussian mixture (GMM).

Initially we have no clue about the distribution. We have to start with a very uniform initial guess.
The algorithm then alternates between:
* Sampling x's from the GMM, and weighting x's by Li, essentially.
* Update the GMM to better match Li using the weighted samples.

In [ ]:
import numpy as np
from matplotlib import pyplot
import matplotlib

In [ ]:
import path_guiding

In [ ]:
def pdf_image(ax, gmm):
    t = np.linspace(-1.,1., 100)
    x,y = np.meshgrid(t,t)
    coord_list = np.vstack((x.ravel(), y.ravel())).T
    pdf_vals = gmm.pdf(coord_list)
    pdf_vals = np.reshape(pdf_vals, (t.size,t.size))
    return ax.imshow(pdf_vals, extent=(-1.,1,1.,-1.))

In [ ]:
gmm = path_guiding.GMM2d()

# nice initialization
init_uniform_means_on_cicle = np.array([
    [-0.02081824, -0.00203204],
    [ 0.4037143 , -0.5633242 ],
    [-0.60468113,  0.32800356],
    [-0.12180968,  0.6798401 ],
    [ 0.6943762 , -0.03451705],
    [ 0.4580511 ,  0.52144015],
    [-0.63349193, -0.26706135],
    [-0.18766472, -0.66355205]
])
init_means = np.random.normal(loc=0., scale=0.25, size=(8,2)).astype(np.float32)
sigma_inv = np.full(8, 10.) #np.random.lognormal(mean=3., sigma=0.2, size=8)
init_precisions = np.zeros((8,2,2), dtype=np.float32)
init_precisions[:,0,0] = sigma_inv
init_precisions[:,1,1] = sigma_inv
init_weights = np.full(8, 1./8., dtype=np.float32)

gmm.initialize(init_weights, init_uniform_means_on_cicle, init_precisions)

In [ ]:
def test_set(x, pdf):
    # Input: probe positions and associated pdf values
    # Output: weights for the EM algo. They are the equivalent of Li/pdf, where Li is the incident radiance.
    #         One can think of it as number of photons corrected for the sampling pdf.
    #         The intuition behind this correction is the following:
    #         Say the pdf is low, implying low sample density. Without the correction, the low pdf value
    #         decreases the effective area density of photons $sum Li_i * w_i / n$.
    #         If the pdf matches the function Li exactly, then all our weights will be 1. Which is what we want to have in the end.
    #         This then also means that the samples are indeed distributed according to Li.
    loc1 = (-0.33,0.1)
    loc2 = (0.33,-0.1)
    locs = test_set.locs = np.array([loc1,loc2])
    ws = np.zeros(xs.shape[0])
    for l in locs:
        scale = 0.25
        ws += np.exp(-np.linalg.norm(xs - l[np.newaxis,:], axis=1)**2/scale/scale)
    ws /= pdf
    return ws

In [ ]:
prior_nu = 1.00001; prior_alpha = 2.01; prior_u = 1.e-5; max_iters = 10; maximization_step_every = 50;
gmm.initialize(init_weights, init_uniform_means_on_cicle, init_precisions)

incremental = path_guiding.GMM2dFitIncremental(
    prior_nu = prior_nu, 
    prior_alpha = prior_alpha, 
    prior_u = prior_u, 
    maximization_step_every = maximization_step_every)

for iter in range(max_iters):
    N = maximization_step_every
    xs = gmm.sample(N)
    pdf = gmm.pdf(xs)
    ws = test_set(xs, pdf)
    
    incremental.fit(gmm, xs, ws)
    fig, ax = pyplot.subplots(1,1, figsize=(10,20))
    pdf_image(ax, gmm)
    ax.scatter(*xs.T, marker = 'o', s = 40, c = ws, edgecolor = 'w')
    ax.scatter(*test_set.locs.T, marker = 'x', s = 100, color = 'r')
    ax.scatter(*gmm.means().T, marker='o', s = 40, color='r')
    pyplot.show()